In [ ]:
import os
import glob
from blur.process import process_media, load_config
from data_filtering.video_benchmark import analyze_video

GLOBAL_HOI_SYS = None  # 全局变量，存放预加载模型
def test_visor_pipeline(visor_video_dir):
    # 1. 获取目录下所有 MP4 文件
    video_files = glob.glob(os.path.join(visor_video_dir, "*.mov"))
    
    if not video_files:
        print(f"在 {visor_video_dir} 没找到视频，请检查路径。")
        return

    print(f"=== 步骤 1: 质量筛选 (Quality Filter) ===")
    print(f"正在检测 {len(video_files)} 个视频的模糊和抖动情况...")
    
    # 批量运行 process_media
    # 针对第一人称视角，建议开启 jitter_plot 查看抖动曲线
    quality_results = process_media(
        inputs=video_files,
        enable_jitter_test=True,  #必须测抖动
        enable_blur_test=True,
        enable_exposure_test=True, # 厨房场景可能有过曝/过暗
        verbose=False
    )

    valid_videos = []

    print("\n=== 质量检测结果 ===")
    for res in quality_results:
        video_path = res['video_path']
        fname = os.path.basename(video_path)
        
        if res['keep']:
            print(f"[✅ 通过] {fname} | 清晰度: {res['blur_video']:.1f} | 抖动: {res['is_shake']}")
            valid_videos.append(video_path)
        else:
            # 打印拒绝原因 (模糊? 抖动? 曝光?)
            reason = []
            if res['is_shake']: reason.append("太抖")
            if res.get('exposure_ratio', 0) > 0.1: reason.append("曝光差")
            if res['blur_video'] < 100: reason.append("太糊") # 假设阈值
            print(f"[❌ 剔除] {fname} | 原因: {', '.join(reason)}")

    print(f"\n=== 内容验证 (Content Benchmark) ===")
    print(f"对通过质量检测的 {len(valid_videos)} 个视频进行手部交互检测...")

    for v_path in valid_videos:
        # 运行 analyze_video 检测交互
        content_res = analyze_video(v_path,hoi_sys=GLOBAL_HOI_SYS)
        
        if content_res:
            fname = os.path.basename(v_path)
            ratio = content_res['interaction_ratio'] * 100
            missing = content_res['missing_hand_ratio'] * 100
            
            if content_res['keep']:
                print(f"[🌟 完美可用] {fname} | 交互占比: {ratio:.1f}% | 手部丢失: {missing:.1f}%")
            else:
                print(f"[⚠️ 无交互] {fname} | 交互占比: {ratio:.1f}% (虽然画质好，但没动作)")

# 使用示例
if __name__ == "__main__":
    # 请修改为你的数据集实际路径
    VISOR_PATH = "uploads" 
    test_visor_pipeline(VISOR_PATH)

/usr1/home/s125mdg55_01/miniforge3/envs/filter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== 步骤 1: 质量筛选 (Quality Filter) ===
正在检测 2 个视频的模糊和抖动情况...


2026-01-24 12:03:41,872 - INFO - Screen Recording 2026-01-24 at 11.31.23 | 484x274 | Blur: 214.2 | Jitter: 0.0%/0.0% | Exposure: 0.0% | Keep: True | 0.5s
2026-01-24 12:03:42,390 - INFO - Screen Recording 2026-01-24 at 11.31.43 | 484x274 | Blur: 161.1 | Jitter: 0.5%/0.5% | Exposure: 0.0% | Keep: True | 0.5s
I0000 00:00:1769227422.460318  552935 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1769227422.491741  553673 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.230.02), renderer: NVIDIA RTX A5000/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.



=== 质量检测结果 ===
[✅ 通过] Screen Recording 2026-01-24 at 11.31.23.mov | 清晰度: 214.2 | 抖动: False
[✅ 通过] Screen Recording 2026-01-24 at 11.31.43.mov | 清晰度: 161.1 | 抖动: False

=== 内容验证 (Content Benchmark) ===
对通过质量检测的 2 个视频进行手部交互检测...
🚀 启动基准测试 (Benchmarks): uploads/Screen Recording 2026-01-24 at 11.31.23.mov
⚠️ 未检测到预加载模型，正在初始化新模型...
[Init] Device set to use cuda
[Init] Loading YOLO model: yolov8s.pt...
[Init] Loading Depth Anything V2 (Small) on cuda...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


...

[🌟 完美可用] Screen Recording 2026-01-24 at 11.31.23.mov | 交互占比: 97.1% | 手部丢失: 0.0%
🚀 启动基准测试 (Benchmarks): uploads/Screen Recording 2026-01-24 at 11.31.43.mov
⚠️ 未检测到预加载模型，正在初始化新模型...
[Init] Device set to use cuda
[Init] Loading YOLO model: yolov8s.pt...
[Init] Loading Depth Anything V2 (Small) on cuda...


Device set to use cuda


🚀 启动基准测试 (Benchmarks): uploads/Screen Recording 2026-01-24 at 11.31.43.mov


Scanning: 100%|██████████| 42/42 [00:02<00:00, 14.10it/s]

[🌟 完美可用] Screen Recording 2026-01-24 at 11.31.43.mov | 交互占比: 90.5% | 手部丢失: 7.1%
